In [6]:
import numpy as np
import pandas as pd
import os, sys
sys.path.append('../evaluation/')
sys.path.append('..')
from tqdm import tqdm
from rdkit import Chem
from openbabel import pybel
from collections import defaultdict
import pickle
from sklearn.cluster import DBSCAN, HDBSCAN

In [7]:
from data_processing.paired_data import CombinedSparseGraphDataset, CombinedSparseGraphDataset
from data_processing.ligand import Ligand
from data_processing.utils import ATOM_TYPE_MAPPING, PP_TYPE_MAPPING
from utils_eval import build_pdb_dict

In [4]:
raw_path = '../../data/cleaned_crossdocked_data/raw'

In [19]:
# num_appearance = defaultdict(lambda : 0)
# num_appearance['10']

num_appearance = {k: defaultdict(lambda : 0) for k in PP_TYPE_MAPPING.keys()}
# num_appearance['Linker']['2']

In [46]:
# this is different from the dataset class mtd
def cluster_non_pp(pos, atom_in_pp):
    pos = np.array(pos)
    # print(pos)
    non_pp_atom_positions = []
    non_pp_atom_pos_dict = []

    for i in range(pos.shape[0]):
        if i not in atom_in_pp:
            # dist_i = np.zeros(num_pp)
            # for j in range(num_pp):
            #     dist_i[j] = np.linalg.norm(atom_positions[i] - pp_positions[j])
            non_pp_atom_positions.append(pos[i])
            non_pp_atom_pos_dict.append({'id':i, 'pos':pos[i]})

    non_pp_atom_positions = np.array(non_pp_atom_positions)
    # print(non_pp_atom_positions)
    if non_pp_atom_positions.shape[0] == 1:
        return {0: [non_pp_atom_pos_dict[0]['id']]}

    clustering_model = HDBSCAN(min_cluster_size=2)
    clustering = clustering_model.fit(non_pp_atom_positions)
    non_pp_atom_labels = clustering.labels_
    max_label = np.max(non_pp_atom_labels)

    for i in range(len(non_pp_atom_labels)):
        if non_pp_atom_labels[i] == -1:
            non_pp_atom_labels[i] = max_label + 1
            max_label += 1

    non_pp_groups = np.unique(non_pp_atom_labels)
    # non_pp_group_center_positions = torch.zeros((len(non_pp_groups), 3))
    non_pp_atom_indices = {label: [] for label in non_pp_groups}

    for group in non_pp_groups:
        # nodes: the index in the non_pp_atom_positions matrix
        nodes = np.where(non_pp_atom_labels==group)[0]
        # print(nodes)

        # atoms: the index in the original ligand
        atoms = []
        for node in nodes:
            # print(node)
            atoms.append(non_pp_atom_pos_dict[int(node)]['id'])
        # print(atoms)
        non_pp_atom_indices[group] = atoms
        
        # positions = non_pp_atom_positions[nodes]
        # print(positions.size())
        # center_pos = np.mean(positions, axis=0)
        # print(center_pos)
        # non_pp_group_center_positions[group] = center_pos

    return non_pp_atom_indices

In [47]:
for folder in tqdm(os.listdir(raw_path)[:10]):
    folder_path = os.path.join(raw_path, folder)
    for fn in tqdm(os.listdir(folder_path)):
        if fn.split('.')[-1] != 'sdf':
            continue
        file = os.path.join(folder_path, fn)

        rdmol = Chem.MolFromMolFile(file, removeHs=False, sanitize=True)
        pbmol = next(pybel.readfile("sdf", file))

        
        try:
            rdmol = Chem.AddHs(rdmol)
            ligand = Ligand(pbmol, rdmol, atom_positions=None, conformer_axis=None, filtering=False)
            rdmol = ligand.rdmol_noH
        except Exception as e:
            print(f'Ligand {file} init failed')
            print(e)
            continue

        atom_in_pp = []
        for pp_node in ligand.graph.nodes:
            atom_indices = list([pp_node.atom_indices]) if type(pp_node.atom_indices)==int else list(sorted(pp_node.atom_indices))
            atom_in_pp += atom_indices
            # positions = pp_node.positions.squeeze()
            # index = pp_node.index
            num_nodes = len(atom_indices)

            for pp_type in pp_node.types:
                # print(pp_type, num_nodes)
                num_appearance[pp_type][num_nodes] += 1

        conformer = rdmol.GetConformer()
        atom_positions = conformer.GetPositions()
        num_nodes = rdmol.GetNumAtoms()
        # print(atom_positions)
        
        if len(atom_in_pp) < num_nodes:
            non_pp_atom_indices = cluster_non_pp(atom_positions, atom_in_pp)
            for group, atom_indices in non_pp_atom_indices.items():
                num_nodes = len(atom_indices)
                num_appearance['Linker'][num_nodes] += 1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 506.95it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 907.51it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 732.21it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 671.58it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 758.27it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 1133.98it/s]

100%|█████████████████

In [48]:
num_appearance['Hydrophobic'][1]

1390

In [49]:
num_appearance['Linker']

defaultdict(<function __main__.<dictcomp>.<lambda>()>,
            {1: 417, 2: 55, 3: 8, 4: 4, 6: 27})

In [50]:
# prob_appearance = {k: defaultdict(lambda : 0) for k in PP_TYPE_MAPPING.keys()}
prob_appearance = {}
for k, d in num_appearance.items():
    total_counts = sum(d.values())
    print(k, total_counts)
    prob_appearance[k] = {num: count/total_counts for num, count in d.items()}
    
    # for num, count in d.items():
    #     prob = count/total_counts
        
    #     prob_appearance[k][num] = prob

Linker 511
Hydrophobic 1390
Aromatic 441
Cation 30
Anion 88
HBond_donor 716
HBond_acceptor 433
Halogen 175


In [51]:
prob_appearance

{'Linker': {1: 0.8160469667318982,
  2: 0.10763209393346379,
  3: 0.015655577299412915,
  4: 0.007827788649706457,
  6: 0.05283757338551859},
 'Hydrophobic': {1: 1.0},
 'Aromatic': {6: 0.8798185941043084, 5: 0.12018140589569161},
 'Cation': {1: 0.5333333333333333, 4: 0.4666666666666667},
 'Anion': {3: 0.75, 5: 0.25},
 'HBond_donor': {1: 1.0},
 'HBond_acceptor': {1: 1.0},
 'Halogen': {1: 1.0}}

# Check the appearance counts and probabilities

In [8]:
with open('../../data/cleaned_crossdocked_data/metadata/num_appearance.pkl', 'rb') as f:
    num_appearance_dict = pickle.load(f)
with open('../../data/cleaned_crossdocked_data/metadata/prob_appearance.pkl', 'rb') as f:
    prob_appearance_dict = pickle.load(f)

In [9]:
num_appearance_dict

{'Linker': {1: 434824,
  2: 98050,
  3: 48167,
  4: 21414,
  6: 6978,
  9: 4682,
  5: 26234,
  8: 2323,
  7: 4648,
  12: 84,
  11: 130,
  10: 565,
  13: 2,
  15: 2,
  14: 40},
 'Hydrophobic': {1: 785537},
 'Aromatic': {6: 332244, 5: 40517, 7: 162},
 'Cation': {1: 33689, 4: 6291},
 'Anion': {3: 68115, 5: 22139, 4: 1834, 2: 9},
 'HBond_donor': {1: 539756},
 'HBond_acceptor': {1: 419842},
 'Halogen': {1: 123502}}

In [10]:
prob_appearance_dict

{'Linker': {1: 0.6708766429630498,
  2: 0.15127834443942154,
  3: 0.07431539027652848,
  4: 0.03303900528124195,
  6: 0.010766142656790245,
  9: 0.007223714519789614,
  5: 0.04047563577790703,
  8: 0.0035840856107371365,
  7: 0.0071712569602695704,
  12: 0.00012960102940246212,
  11: 0.0002005730216942866,
  10: 0.0008717212096713225,
  13: 3.085738795296717e-06,
  15: 3.085738795296717e-06,
  14: 6.171477590593434e-05},
 'Hydrophobic': {1: 1.0},
 'Aromatic': {6: 0.890918500602001,
  5: 0.10864709336779979,
  7: 0.00043440603019926364},
 'Cation': {1: 0.8426463231615808, 4: 0.15735367683841922},
 'Anion': {3: 0.7396006384572787,
  5: 0.24038785193871678,
  4: 0.019913786551136303,
  2: 9.77230528681716e-05},
 'HBond_donor': {1: 1.0},
 'HBond_acceptor': {1: 1.0},
 'Halogen': {1: 1.0}}